# Reed-Solomon

https://en.wikiversity.org/wiki/Reed%E2%80%93Solomon_codes_for_coders

Antes de los códigos Reed-Solomon utilizados para el mensaje, es útil introducir un poco más de matemáticas.

Tenemos que definir la suma, resta, multiplicación y división para bytes y siempre producir bytes como resultado, para evitar cualquier desbordamiento.

Ingenuamente, podríamos intentar usar las definiciones normales para estas operaciones y luego modificar por 256 para evitar que los resultados se desborden. Y esto es exactamente lo que estaremos haciendo, y es lo que se llama un campo de Galois 2 ^ 8.

## campos de Galois

Un campo finito es un conjunto de números, y un campo debe tener seis propiedades que rijan la suma, resta, multiplicación y división: cierre, asociativo, conmutativo, distributivo, de identidad e inverso. 

En pocas palabras, el uso de un campo nos permite estudiar la relación entre los números de este campo y aplicar el resultado a cualquier otro campo que siga las mismas propiedades. 

Por ejemplo, el conjunto de reales ℝ es un campo. En otras palabras, los campos matemáticos estudian la estructura de un conjunto de números.

Sin embargo, los enteros ℤ no son un campo, ya que no todas las divisiones están definidas (como 5/4), lo que viola la propiedad inversa multiplicativa (x tal que x * 4 = 5 no existe). 

Una forma sencilla de solucionarlo es hacer módulo usando un número primo, como 257, o cualquier potencia entera positiva de un número primo: de esta manera, tenemos la garantía de que existe x * 4 = 5, ya que simplemente terminaremos.

Módulo de 2 es un campo de Galois muy interesante, dado que una cadena de 8 bits puede expresar un total de 256 = 2 ^ 8 valores, decimos que usamos un campo de Galois de 2 ^ 8 o GF (2 ^ 8).

Aquí definiremos las operaciones matemáticas habituales que estás acostumbrado a hacer con números enteros, pero adaptadas a GF (2 ^ 8), que básicamente hace operaciones habituales pero módulo 2 ^ 8.

Otra forma de considerar el vínculo entre GF (2) y GF (2 ^ 8) es pensar que GF (2 ^ 8) representa un polinomio de 8 coeficientes binarios. 

Por ejemplo, en GF (2 ^ 8), 170 es equivalente a 10101010 

10101010 = 1 * x ^ 7 + 0 * x ^ 6 + 1 * x ^ 5 + 0 * x ^ 4 + 1 * x ^ 3 + 0 * x ^ 2 + 1 * x + 0 

10101010 = x ^ 7 + x ^ 5 + x ^ 3 + x.

Ambas representaciones son equivalentes, es solo que en el primer caso, 170, la representación es decimal, y en el otro caso es binaria, lo que se puede pensar que representa un polinomio por convención.

Esta última es a menudo la representación utilizada en libros académicos y en implementaciones de hardware (debido a puertas lógicas y registros, que funcionan a nivel binario). Para una implementación de software, se puede preferir la representación decimal para un código más claro.

In [160]:
def qr_check_format(fmt):
    g = 0x537 # = 0b10100110111 
    for i in range(4,-1,-1):
        if fmt & (1 << (i+10)):
            #print ("antes",fmt)
            fmt ^= g << i
            #print ("despues",fmt)
        #print("no entro")
    return fmt

In [161]:
encoded_format = qr_check_format(0b11)
print (encoded_format)

3


In [162]:
def hamming_weight(x):
    weight = 0
    while x > 0:
        weight += x & 1
        x >>= 1
    return weight

def qr_decode_format(fmt):
    best_fmt = -1
    best_dist = 15
    for test_fmt in range(0,32):
        test_code = (test_fmt<<10) + qr_check_format(test_fmt<<10)
        test_dist = hamming_weight(fmt ^ test_code)
        if test_dist < best_dist:
            best_dist = test_dist
            best_fmt = test_fmt
        elif test_dist == best_dist:
            best_fmt = -1
    return best_fmt

In [163]:
qr_decode_format(int("000111101011001",2))

3

# Suma y Resta

Tanto la suma como la resta se reemplazan con XOR en el campo de Galois base 2. 

Esto es lógico: la suma módulo 2 es exactamente como un XOR, y la resta módulo 2 es exactamente igual que la suma módulo 2. Esto es posible porque las sumas y restas en este campo de Galois no lleva consigo.

In [164]:
def gf_add(x, y):
    return x ^ y

def gf_sub(x, y):
    return x ^ y # in binary galois field, subtraction is just the same as addition (since we mod 2)

# Multiplicacion

La multiplicación también se basa en la multiplicación de polinomios. Escribiendo las entradas como polinomios y multiplicando usando la distributiva como de costumbre. 

Ejemplo, 10001001 * 00101010 = 1010001111010

In [165]:
def cl_mul(x,y):
    '''Bitwise carry-less multiplication on integers'''
    z = 0
    i = 0
    while (y>>i) > 0:
        if y & (1<<i):
            z ^= x<<i
        i += 1
    return z

In [166]:
bin(cl_mul(0b10001001,0b00101010))


'0b1010001111010'

El problema de esta multiplicacion es que toma mas de 8 bits

# Definición de un campo finito a partir de un polinomio primitivo


Una clase de polinomios llamados "primitivos" son usados para definir un campo finito de GF(2^m).

Un polinomio es primitivo si:
- es irreducible (no puede ser factoreado en polinomios de menor grado) 

por ejemplo  f(x)=1+x+x^3  define los elementos del campo finito GF(2^3).Por lo tanto hay 2^3=8 elementos

encontrando las raices f(x)=0

definimos $\alpha$ como un elemento del campo tal que f($\alpha$)=0

Entonces

f($\alpha$)=1+$\alpha$+$\alpha$^3=0


 $\alpha$^0=1
 
 $\alpha$^1=$\alpha$
 
 $\alpha$^2=$\alpha$^2

 $\alpha$^3=1+$\alpha$  ( la suma y la resta son iguales en modulo 2)
 
 $\alpha$^4=$\alpha$+$\alpha^2$
 
 $\alpha$^5=1+$\alpha$+$\alpha^2$
 
 $\alpha$^6=1+$\alpha^2$
 
 $\alpha$^7=$\alpha$+$\alpha$^3    Volvio a caer dentro del campo finito
 
 los 8 elementos del campo finito GF(8) son {0,$\alpha$^0,$\alpha$^1,$\alpha$^2,$\alpha$^3,$\alpha$^4,$\alpha$^5,$\alpha$^6}
 
 
 Asi la suma y la multiplicacion quedan definidas como
 
 <img src="1.jpeg">
 


# Multiplicacion dentro de GF

Esto es necesario para que la división se comporte bien, que es permanecer en los límites del Campo Galois, pero sin duplicar valores.

La multiplicacion de 2 palabras de codigo se hacen multiplicando y luego dividiendo por el polinomio generador

In [167]:
def gf_mult_noLUT(x, y, prim=0):
    '''Multiplication in Galois Fields without using a precomputed look-up table (and thus it's slower)
    by using the standard carry-less multiplication + modular reduction using an irreducible prime polynomial'''

    ### Define bitwise carry-less operations as inner functions ###
    def cl_mult(x,y):
        '''Bitwise carry-less multiplication on integers'''
        z = 0
        i = 0
        while (y>>i) > 0:
            if y & (1<<i):
                z ^= x<<i
            i += 1
        return z

    def bit_length(n):
        '''Compute the position of the most significant bit (1) of an integer. Equivalent to int.bit_length()'''
        bits = 0
        while n >> bits: bits += 1
        return bits

    def cl_div(dividend, divisor=None):
        '''Bitwise carry-less long division on integers and returns the remainder'''
        # Compute the position of the most significant bit for each integers
        dl1 = bit_length(dividend)
        dl2 = bit_length(divisor)
        # If the dividend is smaller than the divisor, just exit
        if dl1 < dl2:
            return dividend
        # Else, align the most significant 1 of the divisor to the most significant 1 of the dividend (by shifting the divisor)
        for i in range(dl1-dl2,-1,-1):
            # Check that the dividend is divisible (useless for the first iteration but important for the next ones)
            if dividend & (1 << i+dl2-1):
                # If divisible, then shift the divisor to align the most significant bits and XOR (carry-less subtraction)
                dividend ^= divisor << i
        return dividend
    
    ### Main GF multiplication routine ###

    # Multiply the gf numbers
    result = cl_mult(x,y)
    # Then do a modular reduction (ie, remainder from the division) with an irreducible primitive polynomial so that it stays inside GF bounds
    if prim > 0:
        result = cl_div(result, prim)

    return result

In [168]:
a = 0b10001001
b = 0b00101010

In [169]:
bin(gf_mult_noLUT(a, b, 0)) # multiplication only

'0b1010001111010'

In [170]:
bin(gf_mult_noLUT(a, b, 0x11d))

'0b11000011'

In [171]:
gf_exp = [0] * 512 #Lista de elementos # 512. Python 2.6+ puede considerar usar el tipo bytearray
gf_log = [0] * 256
 


def init_tables(prim=0x11d):
    '''Precompute the logarithm and anti-log tables for faster computation later, using the provided primitive polynomial.'''
    # prim is the primitive (binary) polynomial. Since it's a polynomial in the binary sense,
    # it's only in fact a single galois field value between 0 and 255, and not a list of gf values.
    global gf_exp, gf_log
    gf_exp = [0] * 512 # anti-log (exponential) table
    gf_log = [0] * 256 # log table
    # For each possible value in the galois field 2^8, we will pre-compute the logarithm and anti-logarithm (exponential) of this value
    x = 1
    for i in range(0, 255):
        gf_exp[i] = x # compute anti-log for this value and store it in a table
        gf_log[x] = i # compute log at the same time
        x = gf_mult_noLUT(x, 2, prim)

        # If you use only generator==2 or a power of 2, you can use the following which is faster than gf_mult_noLUT():
        #x <<= 1 # multiply by 2 (change 1 by another number y to multiply by a power of 2^y)
        #if x & 0x100: # similar to x >= 256, but a lot faster (because 0x100 == 256)
            #x ^= prim # substract the primary polynomial to the current value (instead of 255, so that we get a unique set made of coprime numbers), this is the core of the tables generation

    # Optimization: double the size of the anti-log table so that we don't need to mod 255 to
    # stay inside the bounds (because we will mainly use this table for the multiplication of two GF numbers, no more).
    for i in range(255, 512):
        gf_exp[i] = gf_exp[i - 255]
    return [gf_log, gf_exp]


In [172]:
init_tables()

[[0,
  0,
  1,
  25,
  2,
  50,
  26,
  198,
  3,
  223,
  51,
  238,
  27,
  104,
  199,
  75,
  4,
  100,
  224,
  14,
  52,
  141,
  239,
  129,
  28,
  193,
  105,
  248,
  200,
  8,
  76,
  113,
  5,
  138,
  101,
  47,
  225,
  36,
  15,
  33,
  53,
  147,
  142,
  218,
  240,
  18,
  130,
  69,
  29,
  181,
  194,
  125,
  106,
  39,
  249,
  185,
  201,
  154,
  9,
  120,
  77,
  228,
  114,
  166,
  6,
  191,
  139,
  98,
  102,
  221,
  48,
  253,
  226,
  152,
  37,
  179,
  16,
  145,
  34,
  136,
  54,
  208,
  148,
  206,
  143,
  150,
  219,
  189,
  241,
  210,
  19,
  92,
  131,
  56,
  70,
  64,
  30,
  66,
  182,
  163,
  195,
  72,
  126,
  110,
  107,
  58,
  40,
  84,
  250,
  133,
  186,
  61,
  202,
  94,
  155,
  159,
  10,
  21,
  121,
  43,
  78,
  212,
  229,
  172,
  115,
  243,
  167,
  87,
  7,
  112,
  192,
  247,
  140,
  128,
  99,
  13,
  103,
  74,
  222,
  237,
  49,
  197,
  254,
  24,
  227,
  165,
  153,
  119,
  38,
  184,
  180,
  124,
  17,
  

In [173]:
def gf_mul(x,y):
    if x==0 or y==0:
        return 0
    return gf_exp[gf_log[x] + gf_log[y]] # should be gf_exp[(gf_log[x]+gf_log[y])%255] if gf_exp wasn't oversized


In [174]:
print((gf_mul(a,b)))
print("en binario ",bin(gf_mul(a,b)))

195
en binario  0b11000011


# División

Otro beneficio de implementar una tabla logarítmica es que puede usar las diferencias de potencia para definir divisiones. En el siguiente código, se agrega 255 para garantizar que la diferencia no sea negativa.



In [175]:
def gf_div(x,y):
    if y==0:
        raise ZeroDivisionError()
    if x==0:
        return 0
    return gf_exp[(gf_log[x] + 255 - gf_log[y]) % 255]


In [176]:
print(bin(gf_div(gf_mul(a,b),a)))
print(bin(b))

0b101010
0b101010


# potenciacion e inversa

In [177]:
def gf_pow(x, power):
    return gf_exp[(gf_log[x] * power) % 255]

def gf_inverse(x):
    return gf_exp[255 - gf_log[x]] # gf_inverse(x) == gf_div(1, x)

# Polinomios

Antes de pasar a los códigos Reed-Solomon, necesitamos definir varias operaciones sobre polinomios cuyos coeficientes son elementos de campo de Galois. Ésta es una fuente potencial de confusión, ya que los elementos mismos se describen como polinomios; mi consejo es que no lo pienses demasiado. A la confusión se suma el hecho de que x todavía se usa como marcador de posición. Esta x no tiene nada que ver con la x mencionada anteriormente, así que no las confunda.

La notación binaria utilizada anteriormente para los elementos de campo de Galois comienza a volverse incómodamente voluminosa en este punto, por lo que cambiaré a hexadecimal.

00000001 x^4 + 00001111 x^3 + 00110110 x^2 + 01111000 x^1 + 01000000= 01 x^4 + 0f x^3 + 36 x^2 + 78 x^1 + 40

En Python, los polinomios estarán representados por una lista de números en orden descendente de potencias de x, por lo que el polinomio anterior se convierte en [0x01, 0x0f, 0x36, 0x78, 0x40]. (En su lugar, podría haberse utilizado el orden inverso; ambas opciones tienen sus ventajas y desventajas).

In [178]:
### Multiplica polinomio por un escalar

def gf_poly_scale(p,x):
    r = [0] * len(p)
    for i in range(0, len(p)):
        r[i] = gf_mul(p[i], x)
    return r

### suma de 2 polinomios

def gf_poly_add(p,q):
    r = [0] * max(len(p),len(q))
    for i in range(0,len(p)):
        r[i+len(r)-len(p)] = p[i]
    for i in range(0,len(q)):
        r[i+len(r)-len(q)] ^= q[i]
    return r

### multiplica 2 polinomios

def gf_poly_mul(p,q):
    '''Multiply two polynomials, inside Galois Field'''
    # Pre-allocate the result array
    r = [0] * (len(p)+len(q)-1)
    # Compute the polynomial multiplication (just like the outer product of two vectors,
    # we multiply each coefficients of p with all coefficients of q)
    for j in range(0, len(q)):
        for i in range(0, len(p)):
            r[i+j] ^= gf_mul(p[i], q[j]) # equivalent to: r[i + j] = gf_add(r[i+j], gf_mul(p[i], q[j]))
                                                         # -- you can see it's your usual polynomial multiplication
    return r

### evalua un polinomio en x

def gf_poly_eval(poly, x):
    '''Evaluates a polynomial in GF(2^p) given the value for x. This is based on Horner's scheme for maximum efficiency.'''
    y = poly[0]
    for i in range(1, len(poly)):
        y = gf_mul(y, x) ^ poly[i]
    return y


# Perspectiva de la teoría de la codificación

Pero primero, ¿por qué tuvimos que aprender sobre campos finitos y polinomios? Porque esta es la idea principal de los códigos de corrección de errores como Reed-Solomon: en lugar de ver un mensaje como una serie de números (ASCII), lo vemos como un polinomio que sigue las reglas muy bien definidas de la aritmética de campos finitos. En otras palabras, al representar los datos usando polinomios y aritmética de campos finitos, agregamos una estructura a los datos. Los valores del mensaje siguen siendo los mismos, pero esta estructura conceptual ahora nos permite operar sobre el mensaje, sobre los valores de sus caracteres, utilizando reglas matemáticas bien definidas. Esta estructura, que siempre conocemos porque está fuera e independiente de los datos, es la que nos permite reparar un mensaje corrupto.

Por lo tanto, incluso si en la implementación de su código puede optar por no representar explícitamente los polinomios y la aritmética de campo finito, estas nociones son esenciales para que funcionen los códigos de corrección de errores, y encontrará que estas nociones subyacen (aunque implícitamente) a cualquier implementación.

¡Y ahora vamos a poner en práctica estas nociones!

# Codificación RS

Al igual que los códigos BCH, los códigos Reed-Solomon se codifican dividiendo el polinomio que representa el mensaje por un polinomio generador irreducible, y luego el resto es el código RS, que simplemente agregaremos al mensaje original.

¿Por qué? Anteriormente dijimos que el principio detrás de los códigos BCH, y la mayoría de los otros códigos de corrección de errores, es usar un diccionario reducido con palabras muy diferentes para maximizar la distancia entre palabras, y que las palabras más largas tienen mayor distancia: aquí es el mismo principio, primero porque alargamos el mensaje original con símbolos adicionales (el resto) lo que aumenta la distancia, y en segundo lugar porque el resto es casi único (gracias al polinomio generador irreducible cuidadosamente diseñado), de modo que puede ser explotado por algoritmos inteligentes para deducir partes de el mensaje original.

En resumen, con una analogía aproximada al cifrado: nuestro polinomio generador es nuestro diccionario de codificación, y la división polinomial es el operador para convertir nuestro mensaje usando el diccionario (el polinomio generador) en un código RS.


Aquí una función que calcula el polinomio generador para un número dado de símbolos de corrección de errores.

In [179]:
def rs_generator_poly(nsym):
    '''Generate an irreducible generator polynomial (necessary to encode a message into Reed-Solomon)'''
    g = [1]
    for i in range(0, nsym):
        g = gf_poly_mul(g, [1, gf_pow(2, i)])
    return g


In [180]:
def gf_poly_div(dividend, divisor):
    '''Fast polynomial division by using Extended Synthetic Division and optimized for GF(2^p) computations
    (doesn't work with standard polynomials outside of this galois field, see the Wikipedia article for generic algorithm).'''
    # CAUTION: this function expects polynomials to follow the opposite convention at decoding:
    # the terms must go from the biggest to lowest degree (while most other functions here expect
    # a list from lowest to biggest degree). eg: 1 + 2x + 5x^2 = [5, 2, 1], NOT [1, 2, 5]

    msg_out = list(dividend) # Copy the dividend
    #normalizer = divisor[0] # precomputing for performance
    for i in range(0, len(dividend) - (len(divisor)-1)):
        #msg_out[i] /= normalizer # for general polynomial division (when polynomials are non-monic), the usual way of using
                                  # synthetic division is to divide the divisor g(x) with its leading coefficient, but not needed here.
        coef = msg_out[i] # precaching
        if coef != 0: # log(0) is undefined, so we need to avoid that case explicitly (and it's also a good optimization).
            for j in range(1, len(divisor)): # in synthetic division, we always skip the first coefficient of the divisior,
                                              # because it's only used to normalize the dividend coefficient
                if divisor[j] != 0: # log(0) is undefined
                    msg_out[i + j] ^= gf_mul(divisor[j], coef) # equivalent to the more mathematically correct
                                                               # (but xoring directly is faster): msg_out[i + j] += -divisor[j] * coef

    # The resulting msg_out contains both the quotient and the remainder, the remainder being the size of the divisor
    # (the remainder has necessarily the same degree as the divisor -- not length but degree == length-1 -- since it's
    # what we couldn't divide from the dividend), so we compute the index where this separation is, and return the quotient and remainder.
    separator = -(len(divisor)-1)
    return msg_out[:separator], msg_out[separator:] # return quotient, remainder.

In [181]:
def rs_encode_msg(msg_in, nsym):
    '''Reed-Solomon main encoding function'''
    gen = rs_generator_poly(nsym)

    # Pad the message, then divide it by the irreducible generator polynomial
    _, remainder = gf_poly_div(msg_in + [0] * (len(gen)-1), gen)
    # The remainder is our RS code! Just append it to our original message to get our full codeword (this represents a polynomial of max 256 terms)
    msg_out = msg_in + remainder
    # Return the codeword
    return msg_out

In [182]:
def rs_encode_msg(msg_in, nsym):
    '''Reed-Solomon main encoding function, using polynomial division (algorithm Extended Synthetic Division)'''
    if (len(msg_in) + nsym) > 255: raise ValueError("Message is too long (%i when max is 255)" % (len(msg_in)+nsym))
    gen = rs_generator_poly(nsym)
    # Init msg_out with the values inside msg_in and pad with len(gen)-1 bytes (which is the number of ecc symbols).
    msg_out = [0] * (len(msg_in) + len(gen)-1)
    # Initializing the Synthetic Division with the dividend (= input message polynomial)
    msg_out[:len(msg_in)] = msg_in

    # Synthetic division main loop
    for i in range(len(msg_in)):
        # Note that it's msg_out here, not msg_in. Thus, we reuse the updated value at each iteration
        # (this is how Synthetic Division works: instead of storing in a temporary register the intermediate values,
        # we directly commit them to the output).
        coef = msg_out[i]

        # log(0) is undefined, so we need to manually check for this case. There's no need to check
        # the divisor here because we know it can't be 0 since we generated it.
        if coef != 0:
            # in synthetic division, we always skip the first coefficient of the divisior, because it's only used to normalize the dividend coefficient (which is here useless since the divisor, the generator polynomial, is always monic)
            for j in range(1, len(gen)):
                msg_out[i+j] ^= gf_mul(gen[j], coef) # equivalent to msg_out[i+j] += gf_mul(gen[j], coef)

    # At this point, the Extended Synthetic Divison is done, msg_out contains the quotient in msg_out[:len(msg_in)]
    # and the remainder in msg_out[len(msg_in):]. Here for RS encoding, we don't need the quotient but only the remainder
    # (which represents the RS code), so we can just overwrite the quotient with the input message, so that we get
    # our complete codeword composed of the message + code.
    msg_out[:len(msg_in)] = msg_in

    return msg_out

In [183]:
msg_in = [ 0x40, 0xd2, 0x75, 0x47, 0x76, 0x17, 0x32, 0x06,0x27, 0x26, 0x96, 0xc6, 0xc6, 0x96, 0x70, 0xec ]

In [184]:
msg = rs_encode_msg(msg_in, 10)

In [185]:
for i in range(0,len(msg)):
    print(hex(msg[i]), end=' ')

0x40 0xd2 0x75 0x47 0x76 0x17 0x32 0x6 0x27 0x26 0x96 0xc6 0xc6 0x96 0x70 0xec 0xbc 0x2a 0x90 0x13 0x6b 0xaf 0xef 0xfd 0x4b 0xe0 

# Decodificacion de RS

La decodificación Reed-Solomon es el proceso que, a partir de un mensaje potencialmente dañado y un código RS, devuelve un mensaje corregido. En otras palabras, la decodificación es el proceso de reparar su mensaje utilizando el código RS previamente calculado.

Aunque solo hay una forma de codificar un mensaje con Reed – Solomon, hay muchas formas diferentes de decodificarlos y, por lo tanto, existen muchos algoritmos de decodificación diferentes.

Sin embargo, generalmente podemos resumir el proceso de decodificación en 5 pasos [2] [3]:

    1) Calculo del polinomio de síndromes. Esto nos permite analizar qué caracteres tienen errores utilizando Berlekamp-Massey (u otro algoritmo), y también comprobar rápidamente si el mensaje de entrada está dañado.
    2) Calculo del polinomio localizador de borrado / error (de los síndromes). Esto lo calcula Berlekamp-Massey, y es un detector que nos dirá exactamente qué personajes están corruptos.
    3) Calculo del polinomio evaluador de borrado / error (a partir de los síndromes y el polinomio localizador de borrado / error). Necesario para evaluar cuánto fueron manipulados los caracteres (es decir, ayuda a calcular la magnitud).
    4) Calculo del polinomio de magnitud de borrado / error (de los 3 polinomios anteriores): este polinomio también se puede llamar polinomio de corrupción, ya que de hecho almacena exactamente los valores que deben restarse del mensaje recibido para obtener el mensaje original correcto ( es decir, con valores correctos para caracteres borrados). En otras palabras, en este punto, extrajimos el ruido y lo almacenamos en este polinomio, y solo tenemos que eliminar este ruido del mensaje de entrada para repararlo.
    5) Repare el mensaje de entrada simplemente restando el polinomio de magnitud del mensaje de entrada.

Describiremos cada uno de esos cinco pasos a continuación.

Además, los decodificadores también se pueden clasificar por el tipo de error que pueden reparar: borrados (sabemos la ubicación de los caracteres corruptos pero no la magnitud), errores (ignoramos tanto la ubicación como la magnitud), o una combinación de errores. y borrados. Describiremos cómo apoyar todos estos.

# calculo del sindrome

La decodificación de un mensaje Reed-Solomon implica varios pasos. El primer paso es calcular el "síndrome" del mensaje. Trate el mensaje como un polinomio y evalúelo en α0, α1, α2, ..., αn. Dado que estos son los ceros del polinomio generador, el resultado debe ser cero si el mensaje recibido no está dañado (esto se puede usar para verificar si el mensaje está dañado y, después de corregir un mensaje dañado, si el mensaje se reparó por completo). En caso contrario, los síndromes contienen toda la información necesaria para determinar la corrección que se debe realizar. Es sencillo escribir una función para calcular los síndromes.

In [186]:
def rs_calc_syndromes(msg, nsym):
    '''Given the received codeword msg and the number of error correcting symbols (nsym), computes the syndromes polynomial.
    Mathematically, it's essentially equivalent to a Fourrier Transform (Chien search being the inverse).
    '''
    # Note the "[0] +" : we add a 0 coefficient for the lowest degree (the constant). This effectively shifts the syndrome, and will shift every computations depending on the syndromes (such as the errors locator polynomial, errors evaluator polynomial, etc. but not the errors positions).
    # This is not necessary, you can adapt subsequent computations to start from 0 instead of skipping the first iteration (ie, the often seen range(1, n-k+1)),
    synd = [0] * nsym
    for i in range(0, nsym):
        synd[i] = gf_poly_eval(msg, gf_pow(2,i))
    return [0] + synd # pad with one 0 for mathematical precision (else we can end up with weird calculations sometimes)

In [187]:
synd = rs_calc_syndromes(msg, 10)
print(synd)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [188]:
### rompemos un byte

msg[0] = 0  # deliberately damage the message
synd = rs_calc_syndromes(msg, 10)
print(synd)
    

[0, 64, 192, 93, 231, 52, 92, 228, 49, 83, 245]


In [189]:
def rs_check(msg, nsym):
    '''Returns true if the message + ecc has no error or false otherwise (may not always catch a wrong decoding or a wrong message, particularly if there are too many errors -- above the Singleton bound --, but it usually does)'''
    return ( max(rs_calc_syndromes(msg, nsym)) == 0 )


rs_check(msg,10)

False

# Corrección de Errores

Es más sencillo corregir errores en el código si ya se conocen las ubicaciones de los errores. Esto se conoce como corrección de borrado. Es posible corregir un símbolo borrado (es decir, un carácter) por cada símbolo de corrección de errores agregado al código. Si no se conocen las ubicaciones de los errores, se necesitan dos símbolos EC para cada error de símbolo (para que pueda corregir dos veces menos errores que borrados).
Ahora que ya tenemos los síndromes, necesitamos calcular el polinomio del localizador. Esto es facil:

In [192]:
def rs_find_errata_locator(e_pos):
    '''Compute the erasures/errors/errata locator polynomial from the erasures/errors/errata positions
       (the positions must be relative to the x coefficient, eg: "hello worldxxxxxxxxx" is tampered to "h_ll_ worldxxxxxxxxx"
       with xxxxxxxxx being the ecc of length n-k=9, here the string positions are [1, 4], but the coefficients are reversed
       since the ecc characters are placed as the first coefficients of the polynomial, thus the coefficients of the
       erased characters are n-1 - [1, 4] = [18, 15] = erasures_loc to be specified as an argument.'''

    e_loc = [1] # just to init because we will multiply, so it must be 1 so that the multiplication starts correctly without nulling any term
    # erasures_loc = product(1 - x*alpha**i) for i in erasures_pos and where alpha is the alpha chosen to evaluate polynomials.
    for i in e_pos:
        e_loc = gf_poly_mul( e_loc, gf_poly_add([1], [gf_pow(2, i), 0]) )
    return e_loc

def rs_find_error_evaluator(synd, err_loc, nsym):
    '''Compute the error (or erasures if you supply sigma=erasures locator polynomial, or errata) evaluator polynomial Omega
       from the syndrome and the error/erasures/errata locator Sigma.'''

    # Omega(x) = [ Synd(x) * Error_loc(x) ] mod x^(n-k+1)
    _, remainder = gf_poly_div( gf_poly_mul(synd, err_loc), ([1] + [0]*(nsym+1)) ) # first multiply syndromes * errata_locator, then do a
                                                                                   # polynomial division to truncate the polynomial to the
                                                                                   # required length

    # Faster way that is equivalent
    #remainder = gf_poly_mul(synd, err_loc) # first multiply the syndromes with the errata locator polynomial
    #remainder = remainder[len(remainder)-(nsym+1):] # then slice the list to truncate it (which represents the polynomial), which
                                                                          # is equivalent to dividing by a polynomial of the length we want

    return remainder


In [193]:
def rs_correct_errata(msg_in, synd, err_pos): # err_pos is a list of the positions of the errors/erasures/errata
    '''Forney algorithm, computes the values (error magnitude) to correct the input message.'''
    # calculate errata locator polynomial to correct both errors and erasures (by combining the errors positions given by the error locator polynomial found by BM with the erasures positions given by caller)
    coef_pos = [len(msg_in) - 1 - p for p in err_pos] # need to convert the positions to coefficients degrees for the errata locator algo to work (eg: instead of [0, 1, 2] it will become [len(msg)-1, len(msg)-2, len(msg) -3])
    err_loc = rs_find_errata_locator(coef_pos)
    # calculate errata evaluator polynomial (often called Omega or Gamma in academic papers)
    err_eval = rs_find_error_evaluator(synd[::-1], err_loc, len(err_loc)-1)[::-1]

    # Second part of Chien search to get the error location polynomial X from the error positions in err_pos (the roots of the error locator polynomial, ie, where it evaluates to 0)
    X = [] # will store the position of the errors
    for i in range(0, len(coef_pos)):
        l = 255 - coef_pos[i]
        X.append( gf_pow(2, -l) )

    # Forney algorithm: compute the magnitudes
    E = [0] * (len(msg_in)) # will store the values that need to be corrected (substracted) to the message containing errors. This is sometimes called the error magnitude polynomial.
    Xlength = len(X)
    for i, Xi in enumerate(X):

        Xi_inv = gf_inverse(Xi)

        # Compute the formal derivative of the error locator polynomial (see Blahut, Algebraic codes for data transmission, pp 196-197).
        # the formal derivative of the errata locator is used as the denominator of the Forney Algorithm, which simply says that the ith error value is given by error_evaluator(gf_inverse(Xi)) / error_locator_derivative(gf_inverse(Xi)). See Blahut, Algebraic codes for data transmission, pp 196-197.
        err_loc_prime_tmp = []
        for j in range(0, Xlength):
            if j != i:
                err_loc_prime_tmp.append( gf_sub(1, gf_mul(Xi_inv, X[j])) )
        # compute the product, which is the denominator of the Forney algorithm (errata locator derivative)
        err_loc_prime = 1
        for coef in err_loc_prime_tmp:
            err_loc_prime = gf_mul(err_loc_prime, coef)
        # equivalent to: err_loc_prime = functools.reduce(gf_mul, err_loc_prime_tmp, 1)

        # Compute y (evaluation of the errata evaluator polynomial)
        # This is a more faithful translation of the theoretical equation contrary to the old forney method. Here it is an exact reproduction:
        # Yl = omega(Xl.inverse()) / prod(1 - Xj*Xl.inverse()) for j in len(X)
        y = gf_poly_eval(err_eval[::-1], Xi_inv) # numerator of the Forney algorithm (errata evaluator evaluated)
        y = gf_mul(gf_pow(Xi, 1), y)
        
        # Check: err_loc_prime (the divisor) should not be zero.
        if err_loc_prime == 0:
            raise ReedSolomonError("Could not find error magnitude")    # Could not find error magnitude

        # Compute the magnitude
        magnitude = gf_div(y, err_loc_prime) # magnitude value of the error, calculated by the Forney algorithm (an equation in fact): dividing the errata evaluator with the errata locator derivative gives us the errata magnitude (ie, value to repair) the ith symbol
        E[err_pos[i]] = magnitude # store the magnitude for this error into the magnitude polynomial

    # Apply the correction of values to get our message corrected! (note that the ecc bytes also gets corrected!)
    # (this isn't the Forney algorithm, we just apply the result of decoding here)
    msg_in = gf_poly_add(msg_in, E) # equivalent to Ci = Ri - Ei where Ci is the correct message, Ri the received (senseword) message, and Ei the errata magnitudes (minus is replaced by XOR since it's equivalent in GF(2^p)). So in fact here we substract from the received message the errors magnitude, which logically corrects the value to what it should be.
    return msg_in


In [194]:
msg[0] = 0
synd = rs_calc_syndromes(msg, 10)
msg = rs_correct_errata(msg, synd, [0]) # [0] is the list of the erasures locations, here it's the first character, at position 0
print(hex(msg[0]))

0x40


# Corrección de errores

En la situación más probable en la que se desconocen las ubicaciones de los errores (lo que generalmente llamamos errores, en oposición a los borrados donde se conocen las ubicaciones), usaremos los mismos pasos que para los borrados, pero ahora necesitamos pasos adicionales para encontrar la ubicación. El algoritmo de Berlekamp-Massey se utiliza para calcular el polinomio del localizador de errores, que podemos utilizar más adelante para determinar la ubicación de los errores:

In [195]:
def rs_find_error_locator(synd, nsym, erase_loc=None, erase_count=0):
    '''Find error/errata locator and evaluator polynomials with Berlekamp-Massey algorithm'''
    # The idea is that BM will iteratively estimate the error locator polynomial.
    # To do this, it will compute a Discrepancy term called Delta, which will tell us if the error locator polynomial needs an update or not
    # (hence why it's called discrepancy: it tells us when we are getting off board from the correct value).

    # Init the polynomials
    if erase_loc: # if the erasure locator polynomial is supplied, we init with its value, so that we include erasures in the final locator polynomial
        err_loc = list(erase_loc)
        old_loc = list(erase_loc)
    else:
        err_loc = [1] # This is the main variable we want to fill, also called Sigma in other notations or more formally the errors/errata locator polynomial.
        old_loc = [1] # BM is an iterative algorithm, and we need the errata locator polynomial of the previous iteration in order to update other necessary variables.
    #L = 0 # update flag variable, not needed here because we use an alternative equivalent way of checking if update is needed (but using the flag could potentially be faster depending on if using length(list) is taking linear time in your language, here in Python it's constant so it's as fast.

    # Fix the syndrome shifting: when computing the syndrome, some implementations may prepend a 0 coefficient for the lowest degree term (the constant). This is a case of syndrome shifting, thus the syndrome will be bigger than the number of ecc symbols (I don't know what purpose serves this shifting). If that's the case, then we need to account for the syndrome shifting when we use the syndrome such as inside BM, by skipping those prepended coefficients.
    # Another way to detect the shifting is to detect the 0 coefficients: by definition, a syndrome does not contain any 0 coefficient (except if there are no errors/erasures, in this case they are all 0). This however doesn't work with the modified Forney syndrome, which set to 0 the coefficients corresponding to erasures, leaving only the coefficients corresponding to errors.
    synd_shift = len(synd) - nsym

    for i in range(0, nsym-erase_count): # generally: nsym-erase_count == len(synd), except when you input a partial erase_loc and using the full syndrome instead of the Forney syndrome, in which case nsym-erase_count is more correct (len(synd) will fail badly with IndexError).
        if erase_loc: # if an erasures locator polynomial was provided to init the errors locator polynomial, then we must skip the FIRST erase_count iterations (not the last iterations, this is very important!)
            K = erase_count+i+synd_shift
        else: # if erasures locator is not provided, then either there's no erasures to account or we use the Forney syndromes, so we don't need to use erase_count nor erase_loc (the erasures have been trimmed out of the Forney syndromes).
            K = i+synd_shift

        # Compute the discrepancy Delta
        # Here is the close-to-the-books operation to compute the discrepancy Delta: it's a simple polynomial multiplication of error locator with the syndromes, and then we get the Kth element.
        #delta = gf_poly_mul(err_loc[::-1], synd)[K] # theoretically it should be gf_poly_add(synd[::-1], [1])[::-1] instead of just synd, but it seems it's not absolutely necessary to correctly decode.
        # But this can be optimized: since we only need the Kth element, we don't need to compute the polynomial multiplication for any other element but the Kth. Thus to optimize, we compute the polymul only at the item we need, skipping the rest (avoiding a nested loop, thus we are linear time instead of quadratic).
        # This optimization is actually described in several figures of the book "Algebraic codes for data transmission", Blahut, Richard E., 2003, Cambridge university press.
        delta = synd[K]
        for j in range(1, len(err_loc)):
            delta ^= gf_mul(err_loc[-(j+1)], synd[K - j]) # delta is also called discrepancy. Here we do a partial polynomial multiplication (ie, we compute the polynomial multiplication only for the term of degree K). Should be equivalent to brownanrs.polynomial.mul_at().
        #print "delta", K, delta, list(gf_poly_mul(err_loc[::-1], synd)) # debugline

        # Shift polynomials to compute the next degree
        old_loc = old_loc + [0]

        # Iteratively estimate the errata locator and evaluator polynomials
        if delta != 0: # Update only if there's a discrepancy
            if len(old_loc) > len(err_loc): # Rule B (rule A is implicitly defined because rule A just says that we skip any modification for this iteration)
            #if 2*L <= K+erase_count: # equivalent to len(old_loc) > len(err_loc), as long as L is correctly computed
                # Computing errata locator polynomial Sigma
                new_loc = gf_poly_scale(old_loc, delta)
                old_loc = gf_poly_scale(err_loc, gf_inverse(delta)) # effectively we are doing err_loc * 1/delta = err_loc // delta
                err_loc = new_loc
                # Update the update flag
                #L = K - L # the update flag L is tricky: in Blahut's schema, it's mandatory to use `L = K - L - erase_count` (and indeed in a previous draft of this function, if you forgot to do `- erase_count` it would lead to correcting only 2*(errors+erasures) <= (n-k) instead of 2*errors+erasures <= (n-k)), but in this latest draft, this will lead to a wrong decoding in some cases where it should correctly decode! Thus you should try with and without `- erase_count` to update L on your own implementation and see which one works OK without producing wrong decoding failures.

            # Update with the discrepancy
            err_loc = gf_poly_add(err_loc, gf_poly_scale(old_loc, delta))

    # Check if the result is correct, that there's not too many errors to correct
    while len(err_loc) and err_loc[0] == 0: del err_loc[0] # drop leading 0s, else errs will not be of the correct size
    errs = len(err_loc) - 1
    if (errs-erase_count) * 2 + erase_count > nsym:
        raise ReedSolomonError("Too many errors to correct")    # too many errors to correct

    return err_loc

Luego, usando el polinomio del localizador de errores, simplemente usamos un enfoque de fuerza bruta llamado sustitución de prueba para encontrar los ceros de este polinomio, que identifica las ubicaciones del error (es decir, el índice de los caracteres que deben corregirse). Existe un algoritmo más eficiente llamado búsqueda de Chien, que evita volver a calcular toda la evaluación en cada paso de la iteración, pero este algoritmo se deja como ejercicio para el lector.

In [196]:
def rs_find_errors(err_loc, nmess): # nmess is len(msg_in)
    '''Find the roots (ie, where evaluation = zero) of error polynomial by brute-force trial, this is a sort of Chien's search
    (but less efficient, Chien's search is a way to evaluate the polynomial such that each evaluation only takes constant time).'''
    errs = len(err_loc) - 1
    err_pos = []
    for i in range(nmess): # normally we should try all 2^8 possible values, but here we optimize to just check the interesting symbols
        if gf_poly_eval(err_loc, gf_pow(2, i)) == 0: # It's a 0? Bingo, it's a root of the error locator polynomial,
                                                                                   # in other terms this is the location of an error
            err_pos.append(nmess - 1 - i)
    # Sanity check: the number of errors/errata positions found should be exactly the same as the length of the errata locator polynomial
    if len(err_pos) != errs:
        # couldn't find error locations
        raise ReedSolomonError("Too many (or few) errors found by Chien Search for the errata locator polynomial!")
    return err_pos

In [205]:
msg = rs_encode_msg(msg_in, 10)

print("originales ",(hex(msg[0]),hex(msg[10]),hex(msg[15]),hex(msg[20])))

### errores
msg[0] = 6
msg[10] = 7
msg[15] = 5
msg[20] = 8

print("rotos ",(hex(msg[0]),hex(msg[10]),hex(msg[15]),hex(msg[20])))


synd = rs_calc_syndromes(msg, 10)
err_loc = rs_find_error_locator(synd, 10)
pos = rs_find_errors(err_loc[::-1], len(msg)) # find the errors locations
print("posiciones de error", pos)

msg = rs_correct_errata(msg, synd, pos)
print("corregido",(hex(msg[0]),hex(msg[10]),hex(msg[15]),hex(msg[20])))


originales  ('0x40', '0x96', '0xec', '0x6b')
rotos  ('0x6', '0x7', '0x5', '0x8')
posiciones de error [20, 15, 10, 0]
corregido ('0x40', '0x96', '0xec', '0x6b')


Es posible que un decodificador Reed-Solomon decodifique tanto borrados como errores al mismo tiempo, hasta un límite (llamado Singleton Bound) de 2 * e + v <= (nk), donde e es el número de errores, v el número de borrados y (nk) el número de caracteres del código RS (llamado nsym en el código). Básicamente, significa que para cada borrado, solo necesita un carácter de código RS para repararlo, mientras que para cada error necesita dos caracteres de código RS (porque necesita encontrar la posición además del valor / magnitud para corregir). 

Para corregir tanto los errores como los borrados, debemos evitar que los borrados interfieran con el proceso de ubicación del error. Esto se puede hacer calculando los síndromes de Forney, de la siguiente manera.

In [206]:
def rs_forney_syndromes(synd, pos, nmess):
    # Compute Forney syndromes, which computes a modified syndromes to compute only errors (erasures are trimmed out). Do not confuse this with Forney algorithm, which allows to correct the message based on the location of errors.
    erase_pos_reversed = [nmess-1-p for p in pos] # prepare the coefficient degree positions (instead of the erasures positions)

    # Optimized method, all operations are inlined
    fsynd = list(synd[1:])      # make a copy and trim the first coefficient which is always 0 by definition
    for i in range(0, len(pos)):
        x = gf_pow(2, erase_pos_reversed[i])
        for j in range(0, len(fsynd) - 1):
            fsynd[j] = gf_mul(fsynd[j], x) ^ fsynd[j + 1]

    # Equivalent, theoretical way of computing the modified Forney syndromes: fsynd = (erase_loc * synd) % x^(n-k)
    # See Shao, H. M., Truong, T. K., Deutsch, L. J., & Reed, I. S. (1986, April). A single chip VLSI Reed-Solomon decoder. In Acoustics, Speech, and Signal Processing, IEEE International Conference on ICASSP'86. (Vol. 11, pp. 2151-2154). IEEE.ISO 690
    #erase_loc = rs_find_errata_locator(erase_pos_reversed, generator=generator) # computing the erasures locator polynomial
    #fsynd = gf_poly_mul(erase_loc[::-1], synd[1:]) # then multiply with the syndrome to get the untrimmed forney syndrome
    #fsynd = fsynd[len(pos):] # then trim the first erase_pos coefficients which are useless. Seems to be not necessary, but this reduces the computation time later in BM (thus it's an optimization).

    return fsynd

In [207]:
def rs_correct_msg(msg_in, nsym, erase_pos=None):
    '''Reed-Solomon main decoding function'''
    if len(msg_in) > 255: # can't decode, message is too big
        raise ValueError("Message is too long (%i when max is 255)" % len(msg_in))

    msg_out = list(msg_in)     # copy of message
    # erasures: set them to null bytes for easier decoding (but this is not necessary, they will be corrected anyway, but debugging will be easier with null bytes because the error locator polynomial values will only depend on the errors locations, not their values)
    if erase_pos is None:
        erase_pos = []
    else:
        for e_pos in erase_pos:
            msg_out[e_pos] = 0
    # check if there are too many erasures to correct (beyond the Singleton bound)
    if len(erase_pos) > nsym: raise ReedSolomonError("Too many erasures to correct")
    # prepare the syndrome polynomial using only errors (ie: errors = characters that were either replaced by null byte
    # or changed to another character, but we don't know their positions)
    synd = rs_calc_syndromes(msg_out, nsym)
    # check if there's any error/erasure in the input codeword. If not (all syndromes coefficients are 0), then just return the message as-is.
    if max(synd) == 0:
        return msg_out[:-nsym], msg_out[-nsym:]  # no errors

    # compute the Forney syndromes, which hide the erasures from the original syndrome (so that BM will just have to deal with errors, not erasures)
    fsynd = rs_forney_syndromes(synd, erase_pos, len(msg_out))
    # compute the error locator polynomial using Berlekamp-Massey
    err_loc = rs_find_error_locator(fsynd, nsym, erase_count=len(erase_pos))
    # locate the message errors using Chien search (or brute-force search)
    err_pos = rs_find_errors(err_loc[::-1] , len(msg_out))
    if err_pos is None:
        raise ReedSolomonError("Could not locate error")    # error location failed

    # Find errors values and apply them to correct the message
    # compute errata evaluator and errata magnitude polynomials, then correct errors and erasures
    msg_out = rs_correct_errata(msg_out, synd, (erase_pos + err_pos)) # note that we here use the original syndrome, not the forney syndrome
                                                                                                                                  # (because we will correct both errors and erasures, so we need the full syndrome)
    # check if the final message is fully repaired
    synd = rs_calc_syndromes(msg_out, nsym)
    if max(synd) > 0:
        raise ReedSolomonError("Could not correct message")     # message could not be repaired
    # return the successfully decoded message
    return msg_out[:-nsym], msg_out[-nsym:] # also return the corrected ecc block so that the user can check()

# El ejemplo funcionando

In [208]:
# Configuration of the parameters and input message
prim = 0x11d
n = 20 # set the size you want, it must be > k, the remaining n-k symbols will be the ECC code (more is better)
k = 11 # k = len(message)
message = "hello world" # input message

# Initializing the log/antilog tables
init_tables(prim)

# Encoding the input message
mesecc = rs_encode_msg([ord(x) for x in message], n-k)
print("Original: %s" % mesecc)

# Tampering 6 characters of the message (over 9 ecc symbols, so we are above the Singleton Bound)
mesecc[0] = 0
mesecc[1] = 2
mesecc[2] = 2
mesecc[3] = 2
mesecc[4] = 2
mesecc[5] = 2
print("Corrupted: %s" % mesecc)

# Decoding/repairing the corrupted message, by providing the locations of a few erasures, we get below the Singleton Bound
# Remember that the Singleton Bound is: 2*e+v <= (n-k)
corrected_message, corrected_ecc = rs_correct_msg(mesecc, n-k, erase_pos=[0, 1, 2])
print("Repaired: %s" % (corrected_message+corrected_ecc))
print(''.join([chr(x) for x in corrected_message]))

Original: [104, 101, 108, 108, 111, 32, 119, 111, 114, 108, 100, 145, 124, 96, 105, 94, 31, 179, 149, 163]
Corrupted: [0, 2, 2, 2, 2, 2, 119, 111, 114, 108, 100, 145, 124, 96, 105, 94, 31, 179, 149, 163]
Repaired: [104, 101, 108, 108, 111, 32, 119, 111, 114, 108, 100, 145, 124, 96, 105, 94, 31, 179, 149, 163]
hello world


En este notebook se han presentado los principios básicos de los códigos Reed-Solomon. El código que se presenta aquí es bastante genérico y se puede usar para cualquier propósito. Son posibles muchas variaciones y refinamientos de estas ideas, ya que la teoría de la codificación es un campo de estudio muy rico.

